# Text Generation with Recurrent Neural Networks (RNNs)

In this assignment, you'll build upon your understanding of RNNs and Keras to develop a word-level text generation model.  Your goal is to train a model that learns the stylistic nuances of a chosen corpus and generates new, original text segments that echo the source material's essence.

**Datasets**

We've provided several intriguing text corpora to get you started:

*   Mark Twain
*   Charles Dickens
*   William Shakespeare

**Feel free to explore!**  If you have a particular passion for another author, genre, or a specific text, you're encouraged to use your own dataset of raw text.

In [18]:
# Check if we have a GPU available
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU available. If you're on Colab, go to Runtime > Change runtime and select a GPU hardware accelerator.")

No GPU available. If you're on Colab, go to Runtime > Change runtime and select a GPU hardware accelerator.


In [19]:
def download_file(url, file_path):
    import requests
    r = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(r.content)

def load_dataset(file_path, fraction=1.0):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    return raw_text[:int(fraction * len(raw_text))]

dataset = 'mark_twain.txt' # Other options are mark_twain.txt, charles_dickens.txt

download_file(f'https://github.com/UofT-DSI/deep_learning/raw/main/assignments/downloaded_books/' + dataset, dataset)

# Load chosen dataset. NOTE: If Colab is running out of memory, change the `fraction` parameter to a value between 0 and 1 to load less data.
text = load_dataset(dataset, fraction=0.1)

I want to go with Mark Twain since he is my favourite author.

In [22]:
dataset = 'downloaded_books/mark_twain.txt'

fraction = 0.1
with open(dataset, 'r', encoding='utf-8') as file:
    raw_text = file.read()
text = raw_text[:int(fraction * len(raw_text))]

text

"\nThe Project Gutenberg EBook of The Prince and The Pauper, Complete by\nMark Twain (Samuel Clemens)\n\nThis eBook is for the use of anyone anywhere at no cost and with almost\nno restrictions whatsoever. You may copy it, give it away or re-use\nit under the terms of the Project Gutenberg License included with this\neBook or online at www.gutenberg.org\n\nTitle: The Prince and The Pauper, Complete\n\nAuthor: Mark Twain (Samuel Clemens)\n\nRelease Date: August 20, 2006 [EBook #1837]\nLast Updated: February 19, 2018\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PRINCE AND THE PAUPER ***\n\nProduced by David Widger. The earliest PG edition was prepared by Les\nBowler\n\n\n\n\nTHE PRINCE AND THE PAUPER\n\nby Mark Twain\n\nThe Great Seal\n\nI will set down a tale as it was told to me by one who had it of his\nfather, which latter had it of HIS father, this last having in like\nmanner had it of HIS father--and so on, back and still back, 

# 1. Data Preparation (Complete or Incomplete)

Before we can begin training an RNN model, we need to prepare the dataset. This involves cleaning the text, tokenizing words, and creating sequences the model can be trained on.

## 1.1 Data Exploration

Print the first 1000 characters of the dataset. Report the dataset's size and the number of unique characters it contains.

In [23]:
# Your code here
def analyze_text(text):
    dataset_size = len(text)
    unique_characters = len(set(text))
    return dataset_size, unique_characters

# Extract the first 1000 characters and analyze
first_1000_chars = text[:1000]
dataset_size, unique_characters = analyze_text(text)

first_1000_chars

'\nThe Project Gutenberg EBook of The Prince and The Pauper, Complete by\nMark Twain (Samuel Clemens)\n\nThis eBook is for the use of anyone anywhere at no cost and with almost\nno restrictions whatsoever. You may copy it, give it away or re-use\nit under the terms of the Project Gutenberg License included with this\neBook or online at www.gutenberg.org\n\nTitle: The Prince and The Pauper, Complete\n\nAuthor: Mark Twain (Samuel Clemens)\n\nRelease Date: August 20, 2006 [EBook #1837]\nLast Updated: February 19, 2018\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PRINCE AND THE PAUPER ***\n\nProduced by David Widger. The earliest PG edition was prepared by Les\nBowler\n\n\n\n\nTHE PRINCE AND THE PAUPER\n\nby Mark Twain\n\nThe Great Seal\n\nI will set down a tale as it was told to me by one who had it of his\nfather, which latter had it of HIS father, this last having in like\nmanner had it of HIS father--and so on, back and still back, 

In [24]:
dataset_size

1267477

In [25]:
unique_characters

90

## 1.2 Text Pre-Processing

To prepare the dataset for training, we need to clean the text and create a numerical representation the model can interpret. Perform the following pre-processing steps:

*   Convert the entire text to lowercase.
*   Use the `Tokenizer` class from the `keras.preprocessing.text` module to tokenize the text. You should fit the tokenizer on the text and then convert the text to a sequence of numbers. You can use the `texts_to_sequences` method to do this.

**Note**:
* You'll need to specify an appropriate size for the vocabulary. The number of words in the list of most common words can serve as a guide - does it seem like a reasonable vocabulary size?
* Some of the words will be excluded from the vocabulary, as they don't appear often enough. It's important to provide a value for `oov_token` when creating the Tokenizer instance, so that these words can be represented as "unknown".

In [27]:

from tensorflow.keras.preprocessing.text import Tokenizer

# Your code here
# Parameters for the tokenizer
VOCAB_SIZE = 5000  # Adjust vocabulary size as per text complexity and model requirements
OOV_TOKEN = "<OOV>"  # Token for out-of-vocabulary words

# Convert text to lowercase
text = text.lower()

# Initialize and fit the tokenizer
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts([text])

print(text[:1000])
print(f"Vocabulary size: {VOCAB_SIZE}")



the project gutenberg ebook of the prince and the pauper, complete by
mark twain (samuel clemens)

this ebook is for the use of anyone anywhere at no cost and with almost
no restrictions whatsoever. you may copy it, give it away or re-use
it under the terms of the project gutenberg license included with this
ebook or online at www.gutenberg.org

title: the prince and the pauper, complete

author: mark twain (samuel clemens)

release date: august 20, 2006 [ebook #1837]
last updated: february 19, 2018

language: english

character set encoding: utf-8

*** start of this project gutenberg ebook prince and the pauper ***

produced by david widger. the earliest pg edition was prepared by les
bowler




the prince and the pauper

by mark twain

the great seal

i will set down a tale as it was told to me by one who had it of his
father, which latter had it of his father, this last having in like
manner had it of his father--and so on, back and still back, three
hundred years and more, the fat

In [28]:
# Convert text to sequences
text_sequences = tokenizer.texts_to_sequences([text])[0]

# Summary of the tokenizer
vocab_size = len(tokenizer.word_index) + 1  # Including the OOV token

# Display results
text_sequences[:10], vocab_size

([2, 163, 151, 929, 4, 2, 84, 3, 2, 627], 14011)

For datasets like Shakespeare's works, most of the important content can likely be captured within 5000 unique words. The style and vocabulary of Shakespeare are relatively compact compared to modern datasets like web text or news articles.

If everything worked, the following line should show you the first 10 words in the vocabulary:

In [29]:
try:
    print(list(tokenizer.word_index.items())[:10])
except AttributeError:
    print("Tokenizer has not been initialized. Possible issue: Complete the relevant section of the assignment to initialize it."
)


[('<OOV>', 1), ('the', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('in', 7), ('was', 8), ('he', 9), ('it', 10)]


## 1.3 Sequence Generation

Now that the text has been tokenized, we need to create sequences the model can be trained on. There are two parts to this:

*   Use the `texts_to_sequences` method from the tokenizer to convert the text to a list of sequences of numbers.
*   Generate the training sequences. Each training sequence should contain `SEQ_LENGTH` token IDs from the text. The target token for each sequence should be the word that follows the sequence in the text.

In [37]:
import numpy as np

# Generate the training sequences
SEQ_LENGTH = 50  # Choose an appropriate sequence length

# Convert text to sequences using tokenizer (assuming tokenizer and text are defined)
text_sequences = tokenizer.texts_to_sequences([text])[0]

# Generate training sequences
sequences = []
for i in range(len(text_sequences) - SEQ_LENGTH):
    seq = text_sequences[i:i + SEQ_LENGTH + 1]  # SEQ_LENGTH tokens + 1 target token
    sequences.append(seq)

# Check if sequences are generated
if sequences:
    # Separate inputs (X) and targets (y)
    X = np.array([seq[:-1] for seq in sequences])  # Inputs: first SEQ_LENGTH tokens
    y = np.array([seq[-1] for seq in sequences])   # Targets: the next token

    # Display sample outputs
    print(f"Number of training sequences: {len(X)}")
    print(f"First training sequence (X): {X[0]}")
    print(f"First target value (y): {y[0]}")
else:
    print("No training sequences generated. Check the size of the dataset or SEQ_LENGTH.")


Number of training sequences: 231709
First training sequence (X): [   2  163  151  929    4    2   84    3    2  627 1216   24  582  776
 2261 2747   22  929   20   16    2  327    4 1866 1443   23   46  826
    3   14  318   46 3108 2513   28  154  930   10  248   10   86   36
 2514  327   10  157    2  518    4    2]
First target value (y): 163


Assuming your sequences are stored in `X` and the corresponding targets in `y`, the following line should print the first training sequence and its target:

In [38]:
if len(X) > 0 and len(y) > 0:
    print(f'Sequence: {X[0]}\nTarget: {y[0]}')
    print(f'Translated back to words: {[tokenizer.index_word[i] for i in X[0]]} -> {tokenizer.index_word[y[0]]}')
else:
    print("Training sequences have not been generated. Possible issue: Complete the relevant section of the assignment to initialize it.")


Sequence: [   2  163  151  929    4    2   84    3    2  627 1216   24  582  776
 2261 2747   22  929   20   16    2  327    4 1866 1443   23   46  826
    3   14  318   46 3108 2513   28  154  930   10  248   10   86   36
 2514  327   10  157    2  518    4    2]
Target: 163
Translated back to words: ['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'prince', 'and', 'the', 'pauper', 'complete', 'by', 'mark', 'twain', 'samuel', 'clemens', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the'] -> project


And the following code will transform y into a one-hot encoded matrix, and split everything into training and validation sets:

In [39]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

# Ensure that tokenizer has been initialized
if tokenizer is not None:
    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # One last thing: let's drop any examples where the target is the OOV token - we don't want our model to predict that (boring!)
    if OOV_TOKEN in tokenizer.word_index:
        mask = y != tokenizer.word_index[OOV_TOKEN]
        X = X[mask]
        y = y[mask]

    # One-hot encode the target token
    y = to_categorical(y, num_classes=VOCAB_SIZE)

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f'X_train shape: {X_train.shape}')
    print(f'y_train shape: {y_train.shape}')
else:
    print("Tokenizer has not been initialized. Please initialize it and load the vocabulary before continuing.")


X_train shape: (171618, 50)
y_train shape: (171618, 5000)


# 2. Model Development (Complete or Incomplete)

With the dataset prepared, it's time to develop the RNN model. You'll need to define the architecture of the model, compile it, and prepare it for training.

## 2.1 Model Architecture

Define the architecture of your RNN model. You can design it however you like, but there are a few features that it's important to include:

*   An embedding layer that learns a dense representation of the input tokens. You'll need to specify the input dimension (the size of the vocabulary) and the output dimension (the size of the dense representation). Remember, you can look at the documentation [here](https://keras.io/api/layers/core_layers/embedding/).
*   At least one recurrent layer. We have learned how to use LSTM layers in class, but you can use other types of recurrent layers if you prefer. You can find the documentation [here](https://keras.io/api/layers/recurrent_layers/lstm/).
*   A dense layer with a softmax activation function. This layer will output a probability distribution over the vocabulary, so that the model can make predictions about the next token.

In [41]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_dim = 128
model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, input_length=SEQ_LENGTH))

# Recurrent Layer
model.add(LSTM(units=100, return_sequences=False))

# Output Layer
model.add(Dense(units=VOCAB_SIZE, activation='softmax'))  # Softmax for probability distribution

# Model Summary
if len(model.layers) > 0:
    model.summary()
else:
    print("No layers have been added to the model. Please complete the assignment by adding the required layers.")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 2.2 Model Compilation


Compile the model with an appropriate loss function and optimizer. You might also want to track additional metrics, such as accuracy.

Give a short explanation of your choice of loss function and optimizer:

Explanation of choices:
- Loss Function: 'sparse_categorical_crossentropy' is chosen because the target values are integer-encoded. 
It is suitable for multi-class classification problems where the output is a probability distribution over classes.
This loss function calculates the negative log likelihood, encouraging the model to maximize the probability of the correct token.

- Optimizer: 'Adam' is selected because it combines the advantages of both RMSProp and SGD with momentum. 
It adapts the learning rate during training, making it robust and effective for a wide range of problems.
This helps the model converge faster and more reliably.

- Metrics: 'accuracy' is tracked to evaluate how often the model predicts the correct next token. 
This is a straightforward and interpretable metric for classification tasks.

In [44]:
# Your code here

# Compile the model
model.compile(
    optimizer='adam',  # Adaptive optimizer with a default learning rate
    loss='sparse_categorical_crossentropy',  # Suitable for integer-encoded targets
    metrics=['accuracy']  # Track accuracy for model evaluation
)


## 2.3 Model Training

Train the model on the training data you've prepared.

* Train your model for 5 epochs with a batch size of 128. Use the validation data for validation.
* Store the training history in a variable called `history`.

In [45]:
# Your code here

# Your code here
epochs = 5
batch_size = 128

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'y_val shape: {y_val.shape}')

X_train shape: (171618, 50)
y_train shape: (171618, 5000)
X_val shape: (42905, 50)
y_val shape: (42905, 5000)


In [46]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

MemoryError: Unable to allocate 3.20 GiB for an array with shape (171618, 5000) and data type float32

Plot the training history to visualize the model's learning progress. Your plot should include the training and validation loss.

In [12]:
import matplotlib.pyplot as plt

# Your code here

# 3. Text Generation (Complete or Incomplete)

Write a method called `generate_text` that uses the trained model to generate new text. The method should take the following parameters:

*   `model`: The trained RNN model.
*   `tokenizer`: The tokenizer used to pre-process the text data.
*   `seed_text`: The seed text the model will use to generate new text.
*   `max_sequence_len`: The maximum length of the sequence used to generate new text.

The method should return the generated text.

An overview of the text generation process you should follow:

1. Tokenize the seed text using the tokenizer we built before.
2. Pad the sequences to the same length as the training sequences - you can use the `pad_sequences` method from the `keras.preprocessing.sequence` module, which is documented [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences).
3. Use the model to predict the next token in the sequence. Remember that the model will output a probability distribution over the vocabulary, so you'll need to use `np.argmax` to find the token with the highest probability.
4. Add the predicted token to the sequence and remove the first token.
5. Repeat steps 3-4 until you have generated the desired number of tokens.
6. Convert the generated token IDs back to words and return the combined result as a single string.

This is a challenging task, so don't hesitate to ask for help if you need it. It's okay if the generated text doesn't make much sense yet - we'll work on improving the model next.
As a bonus, you can make your method generate "gpt-style" by having it print out each word as it's generated, so you can see the text being generated in real time.

In [13]:
from keras.preprocessing.sequence import pad_sequences

def get_predicted_word(model, sequence):
    """
    Get the predicted word from the model.
    This helper function uses a concept we haven't covered in class yet: temperature.
    In essence, a little bit of randomness in what word we predict can make the text more interesting.
    """

    # Use the model to predict the next token in the sequence
    yhat = model.predict(sequence, verbose=0)

    # Get the index of the predicted word, according to the probabilities
    yhat = np.random.choice(range(VOCAB_SIZE), p=yhat.ravel())

    return yhat

def generate_text(model, tokenizer, seed_text, max_sequence_len):
    """
    Generate new text using the trained model.
    You can use the `get_predicted_word` helper function to help you with this.
    """
    
    return None


In [14]:
# Test the text generation function
generate_text(model, tokenizer, 'hamlet', SEQ_LENGTH)

It's likely that the text generated by your model doesn't make much sense yet. This is because the model hasn't been trained for very long, and the training dataset is relatively small. 

# 4. Model Refinement (Complete or Incomplete)

In this last section, you'll work on improving your model. There are many ways to do this, but here are a few ideas to get you started:

* Use pre-trained embeddings: the code below will help you to load pre-trained embeddings through Keras. 
* Experiment with different model architectures, including the number of layers, the number of units in each layer, and the use of dropout layers.
* Train your model for longer. You can also experiment with different batch sizes.

Implement and test out at least one of these ideas. If you have other ideas for improving the model, feel free to try them out as well.

In [15]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

In [16]:
# Load the pre-trained embeddings
embeddings_index = {}
try:
    with open('glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f'Found {len(embeddings_index)} word vectors.')
except FileNotFoundError:
    print("'glove.6B.100d.txt' File not found. Please make sure you have ran the previous cell.")

Found 400000 word vectors.


In [17]:
# Check if VOCAB_SIZE is set
if VOCAB_SIZE is None:
    print("You need to complete the previous parts of your assignment in order for this to work.")
else:
    # Create an embedding matrix
    embedding_matrix = np.zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        if i < VOCAB_SIZE:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

You need to complete the previous parts of your assignment in order for this to work.


In [18]:
# Check if VOCAB_SIZE is set
if VOCAB_SIZE is None:
    print("You need to complete the previous parts of your assignment in order for this to work.")
else:
    embedding_layer = Embedding(
        VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=SEQ_LENGTH, trainable=False
    )

You need to complete the previous parts of your assignment in order for this to work.


In [19]:
# Define a new model and train it

# Your code here

## Criteria

|Criteria|Complete|Incomplete|
|----|----|----|
|Task 1|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 2|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 3|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 4|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|

## Submission Information

🚨**Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)**🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/deep_learning/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.